In [4]:
from catboost import CatBoostRegressor
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date
from catboost.datasets import titanic

%matplotlib inline

plt.style.use('default')

sns.set(style="whitegrid")

pd.options.display.float_format = '{:20,.2f}'.format

In [5]:
installs = pd.read_csv('TP 1er Cuatrimestre 2019/installs.csv', low_memory=False, parse_dates = ['created'], index_col=['created'])
installs = installs.sort_index()
installs.head()

,application_id,ref_type,ref_hash,click_hash,attributed,implicit,device_countrycode,device_brand,device_model,session_user_agent,user_agent,event_uuid,kind,wifi,trans_id,ip_address,device_language
created,,,,,,,,,,,,,,,,,
2019-04-18 00:00:01.560,70,1891515180541284343,4432995619177048534,NaN,False,False,6287817205707153877,nan,nan,Apsalar-Postback,NaN,NaN,NaN,NaN,NaN,8409380583076227678,nan
2019-04-18 00:00:01.851,70,1891515180541284343,5904733559638204455,NaN,False,False,6287817205707153877,nan,nan,Apsalar-Postback,NaN,NaN,NaN,NaN,NaN,806493034070542041,nan
2019-04-18 00:00:05.152,65,1891515180541284343,896373747754111825,NaN,False,True,6287817205707153877,"308,305,860,557,778,688.00","5,274,185,305,862,703,104.00",http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 9; SM-G9650 Bu...,8c8af5e3-96e7-4a49-9f17-cafa7f300f2c,af_app_opened,False,NaN,6319394158544413414,"6,977,049,253,562,486,784.00"
2019-04-18 00:00:05.589,27,1494519392962156891,3399210824535017892,NaN,False,False,6287817205707153877,nan,"6,794,880,020,077,884,416.00",http-kit/2.0,trivago/216 CFNetwork/978.0.7 Darwin/18.5.0,NaN,NaN,True,NaN,4206683315615624585,"5,221,862,722,669,225,984.00"
2019-04-18 00:00:06.795,339,1494519392962156891,1541425881979513687,NaN,False,False,6287817205707153877,nan,"6,794,880,020,077,884,416.00",http-kit/2.0,TikTok/109005 CFNetwork/758.5.3 Darwin/15.6.0,NaN,NaN,True,NaN,8335834010411202437,"7,528,973,756,559,112,192.00"


In [3]:
installs.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 481511 entries, 2019-04-18 00:00:01.560000 to 2019-04-26 23:59:58.788000
Data columns (total 17 columns):
application_id        481511 non-null int64
ref_type              481511 non-null int64
ref_hash              481511 non-null int64
click_hash            1142 non-null object
attributed            481511 non-null bool
implicit              481511 non-null bool
device_countrycode    481511 non-null int64
device_brand          276443 non-null float64
device_model          454619 non-null float64
session_user_agent    466672 non-null object
user_agent            330768 non-null object
event_uuid            103168 non-null object
kind                  103168 non-null object
wifi                  294829 non-null object
trans_id              8933 non-null object
ip_address            481511 non-null int64
device_language       453934 non-null float64
dtypes: bool(2), float64(3), int64(5), object(7)
memory usage: 59.7+ MB


## Categorizo

In [32]:
device_model = list(installs['device_model'].drop_duplicates())

def categorizarDeviceModel(x):
    return device_model.index(x) if (x != -1) else x

installs['device_model'] = installs['device_model'].fillna(-1).apply(categorizarDeviceModel)

In [33]:
marcas = list(installs['device_brand'].drop_duplicates())

def categorizarDeviceBrand(x):
    return marcas.index(x) if (x != -1) else x

installs['device_brand'] = installs['device_brand'].fillna(-1).apply(categorizarDeviceBrand)

In [34]:
device_language = list(installs['device_language'].drop_duplicates())

def categorizarDeviceLanguage(x):
    return device_language.index(x) if (x != -1) else x

installs['device_language'] = installs['device_language'].fillna(-1).apply(categorizarDeviceLanguage)
installs.describe()

,application_id,ref_type,ref_hash,device_countrycode,device_brand,device_model,ip_address,device_language
count,"481,511.00","481,511.00","481,511.00","481,511.00","481,511.00","481,511.00","481,511.00","481,511.00"
mean,114.63,"1,823,409,079,062,006,528.00","4,608,269,779,645,287,936.00","6,287,817,205,671,257,088.00",6.91,346.91,"4,600,401,451,358,105,088.00",2.70
std,79.54,"149,664,427,496,563,488.00","2,664,214,826,414,426,624.00","35,896,357.27",43.23,742.45,"2,668,896,755,869,683,712.00",4.83
min,1.00,"1,494,519,392,962,156,800.00","40,621,409,780,134.00","6,287,817,205,707,153,408.00",-1.00,-1.00,"33,114,277,486,038.00",-1.00
25%,36.00,"1,891,515,180,541,284,352.00","2,302,034,075,654,886,656.00","6,287,817,205,707,153,408.00",-1.00,19.00,"2,285,331,530,900,604,416.00",1.00
50%,121.00,"1,891,515,180,541,284,352.00","4,605,906,598,492,731,392.00","6,287,817,205,707,153,408.00",1.00,108.00,"4,582,320,595,070,138,880.00",1.00
75%,155.00,"1,891,515,180,541,284,352.00","6,914,483,780,916,173,824.00","6,287,817,205,707,153,408.00",3.00,320.00,"6,906,963,104,056,018,944.00",4.00
max,360.00,"1,891,515,180,541,284,352.00","9,223,365,873,867,066,368.00","6,287,817,205,707,153,408.00","1,084.00","6,472.00","9,223,362,640,454,048,768.00",223.00


In [35]:
info1 = installs['2019-04-21':'2019-04-23'].reset_index().groupby('ref_hash').agg({'created' : 'min'})
info1['created'] = info1['created'] - pd.Timestamp('2019-04-21 00:00:00')
info1['created'] = info1['created'].transform(lambda x: x.total_seconds())
info1 = info1.reset_index()
info1 = info1.merge(installs['2019-04-18':'2019-04-20'].reset_index(),on="ref_hash", how="right")
info1 = info1.rename(columns={'created_x':'label','created_y':'created'})

label1 = info1['label'].to_frame()
train1 = info1.drop('label', axis=1)

In [37]:
info2 = installs['2019-04-22':'2019-04-24'].reset_index().groupby('ref_hash').agg({'created' : 'min'})
info2['created'] = info2['created'] - pd.Timestamp('2019-04-22 00:00:00')
info2['created'] = info2['created'].transform(lambda x: x.total_seconds())
info2 = info2.reset_index()
info2 = info2.merge(installs['2019-04-19':'2019-04-21'].reset_index(),on="ref_hash", how="right")
info2 = info2.rename(columns={'created_x':'label','created_y':'created'})

label2 = info2['label'].to_frame()
train2 = info2.drop('label', axis=1)



In [6]:
info3 = installs['2019-04-23':'2019-04-25'].reset_index().groupby('ref_hash').agg({'created' : 'min'})
info3['created'] = info3['created'] - pd.Timestamp('2019-04-23 00:00:00')
info3['created'] = info3['created'].transform(lambda x: x.total_seconds())
info3 = info3.reset_index()
info3 = info3.merge(installs['2019-04-20':'2019-04-22'].reset_index(),on="ref_hash", how="right")
info3 = info3.rename(columns={'created_x':'label','created_y':'created'})

label3 = info3['label'].to_frame()
train3 = info3.drop('label', axis=1)

In [21]:
info4 = installs['2019-04-24':'2019-04-26'].reset_index().groupby('ref_hash').agg({'created' : 'min'})
info4['created'] = info4['created'] - pd.Timestamp('2019-04-24 00:00:00')
info4['created'] = info4['created'].transform(lambda x: x.total_seconds())
info4 = info4.reset_index()
info4 = info4.merge(installs['2019-04-21':'2019-04-23'].reset_index(),on="ref_hash", how="right")
info4 = info4.rename(columns={'created_x':'label','created_y':'created'})

label4 = info4['label'].to_frame()
train4 = info4.drop('label', axis=1)


In [40]:
info5 = installs['2019-04-25':'2019-04-27'].reset_index().groupby('ref_hash').agg({'created' : 'min'})
info5['created'] = info5['created'] - pd.Timestamp('2019-04-25 00:00:00')
info5['created'] = info5['created'].transform(lambda x: x.total_seconds())
info5 = info5.reset_index()
info5 = info5.merge(installs['2019-04-22':'2019-04-24'].reset_index(),on="ref_hash", how="right")
info5 = info5.rename(columns={'created_x':'label','created_y':'created'})

label5 = info5['label'].to_frame()
train5 = info5.drop('label', axis=1)

In [ ]:
'''train1=train1.fillna(0)
train2=train2.fillna(0)
train3=train3.fillna(0)
train4=train4.fillna(0)
train5=train5.fillna(0)
label1=label1.fillna(60*60*24*3)
label2=label2.fillna(60*60*24*3)
label3=label3.fillna(60*60*24*3)
label4=label4.fillna(60*60*24*3)
label5=label5.fillna(60*60*24*3)'''

'train1=train1.fillna(0)\ntrain2=train2.fillna(0)\ntrain3=train3.fillna(0)\ntrain4=train4.fillna(0)\ntrain5=train5.fillna(0)\nlabel1=label1.fillna(60*60*24*3)\nlabel2=label2.fillna(60*60*24*3)\nlabel3=label3.fillna(60*60*24*3)\nlabel4=label4.fillna(60*60*24*3)\nlabel5=label5.fillna(60*60*24*3)'

In [41]:
train1.drop(['created','ref_type','click_hash','session_user_agent','user_agent','kind','wifi','trans_id','event_uuid'],axis =1, inplace = True )
train2.drop(['created','ref_type','click_hash','session_user_agent','user_agent','kind','wifi','trans_id','event_uuid'],axis =1, inplace = True )
train3.drop(['created','ref_type','click_hash','session_user_agent','user_agent','kind','wifi','trans_id','event_uuid'],axis =1, inplace = True )
train4.drop(['created','ref_type','click_hash','session_user_agent','user_agent','kind','wifi','trans_id','event_uuid'],axis =1, inplace = True )
train5.drop(['created','ref_type','click_hash','session_user_agent','user_agent','kind','wifi','trans_id','event_uuid'],axis =1, inplace = True )
train1 = train1.dropna()
train2 = train2.dropna()
train3 = train3.dropna()
train4 = train4.dropna()
train5 = train5.dropna()

# Analisis

## Ventana 1

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_validation, y_train, y_validation = train_test_split(train1, label1, train_size=0.75, random_state=42)

X_test = train4

categorical_features_indices = ['ref_hash','application_id','attributed','implicit','device_countrycode','device_brand','device_model','ip_address','device_language']

from catboost import CatBoostRegressor
# Initialize data

train_data = train1

eval_data = train5

train_labels = label1
# Initialize CatBoostRegressor
model = CatBoostRegressor(iterations=20,
                             learning_rate=0.02,
                             depth=12,
                             eval_metric='RMSE',
                             random_seed = 23,
                             bagging_temperature = 0.2,
                             od_type='Iter',
                             metric_period = 1,
                             od_wait=100)
# Fit model
model.fit(X_train, y_train,
        cat_features=categorical_features_indices,
          eval_set=(X_validation, y_validation),
          plot=True)
# Get predictions
preds = model.predict(eval_data)

In [ ]:
X = train1
y = label1
model.score(X,y)


In [ ]:
preds = pd.DataFrame(preds).reset_index()
np.sqrt(np.sum(((label5['label']-preds[0])**2)/len(label5['label'])))

In [ ]:
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor, Pool
from catboost.datasets import msrank
from sklearn.model_selection import train_test_split

##########################################
######### SIMPLE BASELINE MODEL ##########
##########################################

# Prepare simple baselines (just mean target on first part of train pool).
baseline_value = label1.mean()
train_baseline = np.array([baseline_value] * y_validation.shape[0])
test_baseline = np.array([baseline_value] * label4.shape[0])

# Create pools
train_pool = Pool(X_validation, y_validation, baseline=train_baseline)
test_pool = Pool(train4, label4, baseline=test_baseline)

# Train CatBoost model
model.fit(train_pool, eval_set=test_pool)

# Apply model on pool with baseline values
preds1 = model.predict(test_pool)

# Apply model on numpy.array and then add the baseline values
preds2 = test_baseline + model.predict(train4)

# Check that preds have small diffs
assert (np.abs(preds1 - preds2) < 1e-6).all()

print(mean_squared_error(label4, preds1))

##########################################
######### LINEAR BASELINE MODEL ##########
##########################################

# Train baseline model (linear regression) on first part of train pool
baseline_model = Ridge(alpha=3e3, normalize=True)
baseline_model.fit(X_train, y_train)

# Prepare baselines
train_baseline = baseline_model.predict(X_validation)
test_baseline = baseline_model.predict(train4)

# Create pools
train_pool = Pool(X_validation, y_validation, baseline=train_baseline)
test_pool = Pool(train4, label4, baseline=test_baseline)

# Train CatBoost model
model.fit(train_pool, eval_set=test_pool)

# Apply model on pool with baseline values
preds1 = catboost_model.predict(test_pool)

# Apply model on numpy.array and then add the baseline values
preds2 = baseline_model.predict(train4) + model.predict(train4)

# Check that preds have small diffs
assert (np.abs(preds1 - preds2) < 1e-6).all()

print(mean_squared_error(label4, preds1))

## Ventana 2

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_validation, y_train, y_validation = train_test_split(train2, label2, train_size=0.75, random_state=42)

X_test = train5

categorical_features_indices = ['ref_hash','application_id','attributed','implicit','device_countrycode','device_brand','device_model','ip_address','device_language']

# Fit model
model.fit(X_train, y_train,
        cat_features=categorical_features_indices,
          eval_set=(X_validation, y_validation),
          plot=True)
# Get predictions
preds = model.predict(eval_data)

In [ ]:
X = train2
y = label2
model.score(X,y)


In [ ]:
preds = pd.DataFrame(preds).reset_index()
label = label1.reset_index()
diferencia = label.merge(preds,on='index')
diferencia['dif'] = diferencia[0] - diferencia['label']
diferencia.agg({'dif':"mean"})


# Prueba con todos los datos

In [7]:
target = pd.read_csv('TP 1er Cuatrimestre 2019/target_competencia_ids.csv', low_memory=False)
target.head()

,ref_hash,obj
0,1000169251625791246_sc,0
1,1000169251625791246_st,0
2,1000395625957344683_sc,0
3,1000395625957344683_st,0
4,1003027494996471685_sc,0


In [16]:
installs['sc'] = installs['ref_hash'].transform(lambda x: str(x)+'_sc')

mergeados = installs.merge(target, left_on = 'sc',right_on='ref_hash', how='inner')

In [18]:
len(mergeados)

5471

In [19]:
len(target)

8074

In [22]:
train4['sc'] = train4['ref_hash'].transform(lambda x: str(x)+'_sc')
mergeados = train4.merge(target, left_on = 'sc',right_on='ref_hash', how='inner')

In [23]:
len(mergeados)

1780

In [24]:
len(train4)

160617